In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

filelist = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith(".in"):
            filelist.append(os.path.join(dirname, filename))

for filename in filelist:
    
    print('Process file: ' + filename)

    # Part 0: Reading Input File
    with open(filename, 'r') as data:
        first_line = data.readline()
        rows, cols, num_drones, num_turns, payload = first_line.split()
        rows, cols, num_drones, num_turns, payload = int(rows), int(cols), int(num_drones), int(num_turns), int(payload)
    
        second_line = data.readline()
        num_prodtypes = int(second_line)
        
        third_line = data.readline()
        prodtype_weights = [int(x) for x in third_line.split()]
        
        fourth_line = data.readline()
        num_warehouses = int(fourth_line)
        
        # 2D Array, row = warehouse, col = (x, y) coordinates
        warehouse_locs = []
        # 2D Array, row = warehouse, col = quantity of each prod
        warehouse_prods = []
        for i in range(num_warehouses):
            wh_coords = data.readline()
            warehouse_locs.append([int(x) for x in wh_coords.split()])
            prod_qty = data.readline()
            warehouse_prods.append([int(x) for x in prod_qty.split()])
        
        num_orders = data.readline()
        num_orders = int(num_orders)
        
        # 2D Array, row = order, col = (x, y) coordinates
        order_locs = []
        # 2D Array, row = warehouse, col = quantity of each prod type ordered
        order_prods = []
        for i in range(num_orders):
            order_coords = data.readline()
            order_locs.append([int(x) for x in order_coords.split()])
            num_prods_ordered = data.readline()
            num_prods_ordered = int(num_prods_ordered)
            
            prods_ordered = [0] * num_prodtypes
                        
            prodtype_ordered = data.readline()
            prodtype_ordered = [int(x) for x in prodtype_ordered.split()]
            
            for prodtype in prodtype_ordered:
                prods_ordered[prodtype] += 1
            
            order_prods.append(prods_ordered)
        
        prodtype_weights = np.array(prodtype_weights)
        warehouse_locs = np.array(warehouse_locs)
        warehouse_prods = np.array(warehouse_prods)
        order_locs = np.array(order_locs)
        order_prods = np.array(order_prods)
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def recalcDroneLocTime(partialSoln, orig_avail_time, orig_drone_loc, warehouse_locs, order_locs):
    
    avail_time = orig_avail_time
    drone_loc = orig_drone_loc
    
    for command in partialSoln:
        drone_id = command[0]
        if (command[1] == 'L') or (command[1] == 'U'):
            wh_id = command[2]
            curr_wh_loc = warehouse_locs[wh_id]
            dist_wh = np.linalg.norm(curr_wh_loc - drone_loc, 2)
            avail_time += (np.ceil(dist_wh) + 1)
            drone_loc = warehouse_locs[wh_id]
        elif command[1] == 'D':
            order_id = command[2]
            curr_order_loc = order_locs[order_id]
            dist_order = np.linalg.norm(curr_order_loc - drone_loc, 2)
            avail_time += (np.ceil(dist_order) + 1)
            drone_loc = order_locs[order_id]
    
    return avail_time, drone_loc

def partialSolnStrOutput(list_commands):    
    convertedPartialSoln = []
    for command in list_commands:
        if command[1] == 'W':
            convertedPartialSoln.append(str(command[0]) + " W " + str(command[2]))
        else:
            convertedPartialSoln.append(str(command[0]) + " " + str(command[1]) + " " + str(command[2]) + " " + str(command[3]) + " " + str(command[4]))
    
    return convertedPartialSoln
            

In [ ]:
dist_order_warehouse = order_locs - warehouse_locs.reshape(warehouse_locs.shape[0], 1, warehouse_locs.shape[1])
dist_order_warehouse = np.sqrt(np.einsum('ijk, ijk->ij', dist_order_warehouse, dist_order_warehouse))
avg_dist_order_warehouse = np.mean(dist_order_warehouse, axis = 0)

In [ ]:
solution = [] # List of string

# Drone's timing
drone_avail_time = np.array([0] * num_drones)
drone_curr_loc = np.tile(warehouse_locs[0], (num_drones, 1))

mean_order_loc = np.mean(order_locs, axis = 0)

skip_counter = 0
# Any drone still available before total number of turns
while (np.any(drone_avail_time < num_turns) and (np.sum(order_prods) > 0) and (skip_counter < len(drone_avail_time))):
    
    # Choose the earliest available drone, skipping those without solutions
    drone_no = np.argsort(drone_avail_time)[skip_counter]
    # No drone to plan anymore
    if drone_avail_time[drone_no] >= num_turns:
        break
    
    print("Planning for drone " + str(drone_no) + " available at time " + str(drone_avail_time[drone_no]))
    
    # Item Planning Phase    
    avail_time = drone_avail_time[drone_no]
    drone_loc_plan = drone_curr_loc[drone_no]
    # Record available space in the drone
    avail_space = payload
    # Current drone solution
    drone_solution = []
    
    order_prods_plan = np.copy(order_prods)
    warehouse_prods_plan = np.copy(warehouse_prods)

    # Aggregate weights per order
    order_weights = order_prods_plan * prodtype_weights # 2D matrix, row = order, col = prodtype
    weights_per_order = np.sum(order_weights, axis = 1) # 1D vector, length = number of orders
    
    dist_drone_order = np.linalg.norm(order_locs - drone_loc_plan, 2, axis = 1) # 1D vector, length = number of orders
    
    dist_mean_order = np.linalg.norm(order_locs - mean_order_loc, 2, axis = 1)
    
    score_per_order = 2/3 * weights_per_order / np.max(weights_per_order) + 1/3 * dist_drone_order / np.max(dist_drone_order)
    
    load_items = dict() # key = prodtype, val = units
    deliver_items = dict() # key = orderNo, val = load_dict
    
    # Phase 1: Order and Product Assignment
    for orderNo in weights_per_order.argsort():
    #for orderNo in score_per_order.argsort():        
        
        # Filter to the ordered product types
        ordered_prodtypes = np.where(order_prods_plan[orderNo] > 0)[0]
        ordered_prod_unitweight = prodtype_weights[ordered_prodtypes]
                
        # Heaviest item first
        for idx in ordered_prod_unitweight.argsort()[::-1]:
            
            # If available space is less than the unit weight of this product
            if avail_space < ordered_prod_unitweight[idx]:
                continue
            
            units_to_load = avail_space // ordered_prod_unitweight[idx]
            
            if units_to_load > order_prods_plan[orderNo, ordered_prodtypes[idx]]:
                units_to_load = order_prods_plan[orderNo, ordered_prodtypes[idx]]
                order_prods_plan[orderNo, ordered_prodtypes[idx]] = 0
            else:
                order_prods_plan[orderNo, ordered_prodtypes[idx]] -= units_to_load
            
            avail_space -= (units_to_load * ordered_prod_unitweight[idx])
            
            if units_to_load > 0:
                if orderNo in deliver_items:
                    if ordered_prodtypes[idx] in deliver_items[orderNo]:
                        deliver_items[orderNo][ordered_prodtypes[idx]] += units_to_load
                    else:
                        deliver_items[orderNo][ordered_prodtypes[idx]] = units_to_load
                else:
                    deliver_items[orderNo] = {ordered_prodtypes[idx]: units_to_load}
                
                if ordered_prodtypes[idx] in load_items:
                    load_items[ordered_prodtypes[idx]] += units_to_load
                else:
                    load_items[ordered_prodtypes[idx]] = units_to_load
    
    # Phase 2: Warehouse & Customer delivery route planning
    # Warehouse route planning
    while ((len(load_items) > 0) and (avail_time < num_turns)):
        # Distance from drone to warehouse
        dist_warehouse = np.linalg.norm(warehouse_locs - drone_loc_plan, 2, axis = 1)
        
        for warehouse_id in dist_warehouse.argsort():
            # Available product types in this warehouse
            warehouse_avail_prods = np.where(warehouse_prods_plan[warehouse_id] > 0)[0]
            # Remaining product types to load
            target_items = np.array(list(load_items.keys()))
            
            warehouse_load_prodtypes = target_items[np.in1d(target_items, warehouse_avail_prods)]
            
            if len(warehouse_load_prodtypes) > 0:
                
                for prodtype in warehouse_load_prodtypes:
                    
                    # Load minimum of the required number and available number in the warehouse
                    load_num_units = min(load_items[prodtype], warehouse_prods_plan[warehouse_id, prodtype])
                    
                    load_items[prodtype] -= load_num_units
                    if load_items[prodtype] == 0:
                        del load_items[prodtype]
                    
                    warehouse_prods_plan[warehouse_id, prodtype] -= load_num_units
                    
                    avail_time += (np.ceil(dist_warehouse[warehouse_id]) + 1)
                    drone_loc_plan = warehouse_locs[warehouse_id]
                    
                    #drone_solution.append(str(drone_no) + " L " + str(warehouse_id) + " " + str(prodtype) + " " + str(load_num_units))
                    drone_solution.append([drone_no, "L", warehouse_id, prodtype, load_num_units])
                
                break # Break, have to recalculate distance
    
    # Order Delivery Route Planning
    while ((len(deliver_items) > 0) and (avail_time < num_turns)):
        
        cust_order_no = np.array(list(deliver_items.keys()))
        cust_order_locs = order_locs[cust_order_no]
        
        # Distance from drone to customers
        dist_cust = np.linalg.norm(cust_order_locs - drone_loc_plan, 2, axis = 1)
        
        # Go to nearest order for delivery
        delivery_cust_idx = dist_cust.argmin()
        delivery_order_no = cust_order_no[delivery_cust_idx]
        
        for prodtype, units in deliver_items[delivery_order_no].items():
            
            avail_time += (np.ceil(dist_cust[delivery_cust_idx]) + 1)
            drone_loc_plan = cust_order_locs[delivery_cust_idx]
            
            #drone_solution.append(str(drone_no) + " D " + str(delivery_order_no) + " " + str(prodtype) + " " + str(units))
            drone_solution.append([drone_no, "D", delivery_order_no, prodtype, units])
        
        del deliver_items[delivery_order_no]
    
    # Do heuristic search here
    #print(drone_solution)
    
    # Update avail_time and drone_loc_plan based on the updated partial solution
    #avail_time, drone_loc_plan = recalcDroneLocTime(partialSoln, drone_avail_time[drone_no], 
    #                                                drone_curr_loc[drone_no], warehouse_locs, 
    #                                                order_locs)
    
    # Convert partial solution to list of strings
    drone_solution = partialSolnStrOutput(drone_solution)
        
    if avail_time <= num_turns:
        #print("soln found")
        solution += drone_solution
        drone_curr_loc[drone_no] = drone_loc_plan
        drone_avail_time[drone_no] = avail_time
        # Persist np arrays
        order_prods = order_prods_plan
        warehouse_prods = warehouse_prods_plan
        # Counter to keep track how many rounds of no solution found
        skip_counter = 0
    else:
        skip_counter += 1

if (np.sum(order_prods) > 0):
    wait_drones = np.where(drone_avail_time < num_turns)[0]
    for wait_drone in wait_drones:
        solution.append(str(wait_drone) + " W " + str(num_turns - drone_avail_time[wait_drone]))

    drone_avail_time[wait_drones] = num_turns

In [ ]:
with open('/kaggle/working/submission.csv', 'w') as fd:
    fd.write(str(len(solution)) + '\n')
    for writeLine in solution:
        fd.write(writeLine + '\n')